In [1]:
import pm4py
from pm4py.objects.ocel.obj import OCEL
from typing import Optional
import pandas as pd

In [2]:
ocel = pm4py.read_ocel2_sqlite("../data/event_logs/ContainerLogistics-v1.sqlite")

/home/grkmr/Projects/ocean/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/ocel/importer/sqlite/variants/ocel20.py:70: UserWarning: There are unsatisfied OCEL 2.0 constraints in the given relational database: ['const_12_existence_type_ocel_time_obj_type_spec_tables', 'const_13_existence_type_ocel_time_ev_type_spec_tables', 'const_14_primary_key_object_event_map_type_tables', 'const_15_primary_key_object_event_tables', 'const_16_primary_key_event_object_table', 'const_17_primary_key_object_object_table', 'const_18_primary_key_event_type_spec_tables', 'const_19_foreign_key_event', 'const_20_foreign_key_object', 'const_21_foreign_key_event_object', 'const_22_foreign_key_object_object', 'const_23_foreign_key_event_type_specific', 'const_24_foreign_key_object_type_specific']
  warnings.warn("There are unsatisfied OCEL 2.0 constraints in the given relational database: "+str(unsatisfied))
/home/grkmr/Projects/ocean/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/log/

# Filter

In [19]:
from typing import Literal, Optional
from pydantic import BaseModel

class BaseFilterConfig(BaseModel):
    type: str
    mode: Optional[Literal["include", "exclude"]] = "include"

## Filter Entities by type

In [20]:
def filter_by_entity_types(entity_table:pd.DataFrame,entity_type_field:str,entity_type_list:list[str],include:bool=True):
    eventMask = entity_table[entity_type_field].isin(entity_type_list)
    return eventMask if include else ~eventMask

### Filter by activity name

In [21]:
class EventTypeFilterConfig(BaseFilterConfig):
    type: Literal["event_type"]
    event_types: list[str]

In [22]:
def filter_by_event_types(ocel:OCEL,event_types:list[str],include:bool=True):
    return filter_by_entity_types(ocel.events, ocel.event_activity, event_types, include)

### Filter by object type

In [23]:
class ObjectTypeFilterConfig(BaseFilterConfig):
    type:Literal["object_type"]
    object_types:list[str]

In [24]:
def filter_by_object_types(ocel:OCEL,object_types:list[str],include:bool=True):
    return filter_by_entity_types(ocel.objects, ocel.object_id_column, object_types, include)

## Filter Entities by frequencies

In [25]:
def filter_by_entity_frequency(entity_tabele:pd.DataFrame, entity_type_field:str, threshold:int):
    cumulative_freq = entity_table.value_counts(entity_type_field, normalize=True).cumsum()
    included_entities = cumulative_freq[cumulative_freq <= threshold].index
    return filter_by_entity_types(entity_table,entity_type_field,included_entities)

### Filter by Activity type frequency 

In [26]:
def filter_by_activity_frequency(ocel:OCEL, threshold:int):
    return filter_by_entity_frequency(ocel.events, ocel.event_activity, threshold)

### Filter by Object type frequency

In [27]:
def filter_by_object_frequency(ocel:OCEL, threshold:int):
    return filter_by_entity_frequency(ocel.objects, ocel.object_id_column, threshold)

## Filter by time range

In [28]:
def filter_by_time_range(ocel:OCEL,start_time:Optional[str]=None, end_time:Optional[str]=None , include:bool=True):
    if start_time is not None:
        start_time = pd.Timestamp(start_time, tz="UTC")
    if end_time is not None:
        end_time = pd.Timestamp(end_time, tz="UTC")
    
    events_df = ocel.events

    mask = pd.Series([True] * len(events_df), index=events_df.index)
    if start_time is not None:
        mask &= events_df["ocel:timestamp"] >= start_time
    if end_time is not None:
        mask &= events_df["ocel:timestamp"] <= end_time

    return mask if include else ~mask

## Filter by Relation Count

### E20

In [137]:
def filter_by_e2o_counts(
    ocel: OCEL,
    event_type: str,
    object_type: str,
    min_relations: int,
    entity_field: Optional[Literal["object", "event"]] = "object",
    max_relations: Optional[int] = None,
    qualifier: Optional[str] = None
):
    target_id = ocel.event_id_column if entity_field == "event" else ocel.object_id_column
    target_dataframe = ocel.events if entity_field =="event" else ocel.objects
    mask = (
        (ocel.relations[ocel.event_activity] == event_type) &
        (ocel.relations[ocel.object_type_column] == object_type)
    )
    
    if qualifier is not None and 'qualifier' in ocel.relations.columns:
        mask &= (ocel.relations['qualifier'] == qualifier)
    
    filtered_relations = ocel.relations[mask]

    entity_counts = (
        filtered_relations
        .groupby(target_id)
        .size()
        .reset_index(name='entity_count')
    )
    
    if max_relations is not None:
        entity_counts = entity_counts[
            (entity_counts['entity_count'] >= min_relations) & 
            (entity_counts['entity_count'] <= max_relations)
        ]
    else:
        entity_counts = entity_counts[entity_counts['entity_count'] >= min_relations]
    
    entity_mask = (target_dataframe[ocel.event_activity if entity_field =="event" else ocel.object_type_column] !=(event_type if entity_field =="event" else object_type) ) |  (target_dataframe[target_id].isin(entity_counts[target_id]))
    
    return entity_mask

In [140]:
    qualifier_col = ocel.qualifier
    activity_col = ocel.event_activity
    object_type_col = ocel.object_type_column
    event_id_col = ocel.event_id_column

    grouped_relations = (
        ocel.relations.groupby(
            [event_id_col, qualifier_col, activity_col, object_type_col]
        )
        .size()
        .reset_index()
        .rename(columns={0: "count"})
    )

    summary: pd.DataFrame = (
        grouped_relations.groupby([qualifier_col, activity_col, object_type_col])[
            "count"
        ]
        .agg(["min", "max"])
        .reset_index()
        .rename(columns={"min": "min_count", "max": "max_count"})
    )

 

In [152]:
def get_e2o_summary(ocel:OCEL, direction:Optional[Literal["event" ,"object"]]="event"):
    qualifier_col = ocel.qualifier
    activity_col = ocel.event_activity
    object_type_col = ocel.object_type_column
    event_id_col = ocel.event_id_column
    object_id_col = ocel.object_id_column

    grouped_relations = (
        ocel.relations.groupby(
            [event_id_col if direction =="event" else object_id_col, qualifier_col, activity_col, object_type_col]
        )
        .size()
        .reset_index()
        .rename(columns={0: "count"})
    )

    return (
        grouped_relations.groupby([qualifier_col, activity_col, object_type_col])[
            "count"
        ]
        .agg(["min", "max","sum"])
        .reset_index()
        .rename(columns={"min": "min_count", "max": "max_count"})
    )
